# A story of failures in the optimisation of Water Distribution Systems.

Aware of the computational needs of our idea (joint optimisation of WDS design and operation), I started the development of a C++ Library to work as an interface between an optimization library and a hydraulic simulation library.



In [ ]:
# Imports
import os
import sys
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as subp
import pandas as pd
import numpy as np

import pygmo as pg

sys.path.append('..')
import pybeme

In [ ]:
# Load the experiments, prepare the plots
experiments = pybeme.load_experiments(os.path.join('data', 'all'), verbose=False)

In [ ]:
# Plot the pareto front of the first round of experiments.
# There is something wrong, so plot the Head and pressure for the tanks, junctions
msize = 10
xlims = [-.5e6, 20.5e6]
ylims = [-0.05, 0.605]
xaxis_title_text='Cost [$]'
yaxis_title_text='OF Reliability Index f1 [-]'
title_font_size=20
legend_font_size=16
def_font_size=12
def fix_layout(a_fig: go.Figure, a_title: str) -> None:
    a_fig.update_layout(title=dict(
                            text=a_title,
                            xanchor='center',
                            x=0.5,
                            yanchor='top',
                            y=1,
                            font_size=title_font_size
                        ),
                        plot_bgcolor='white',
                        paper_bgcolor='white',
                        xaxis=dict(
                            title=xaxis_title_text,
                            range=xlims,
                            automargin=True,
                            showline=True,
                            showgrid=True,
                            linewidth=1,
                            linecolor='grey',
                            zerolinecolor='black',
                            gridcolor='lightgrey'
                        ),
                        yaxis=dict(
                            title=yaxis_title_text,
                            range=ylims,
                            automargin=True,
                            showline=True,
                            showgrid=True,
                            linewidth=1,
                            linecolor='grey',
                            zerolinecolor='black',
                            gridcolor='lightgrey'
                        ),
                        width=1200,
                        height=800,
                        font=dict(
                            family="Lato",
                            color="black",
                            size=def_font_size
                        ),
                        margin=dict(
                            l=10,
                            r=10,
                            b=10,
                            t=50,
                            pad=0
                        ),
                        showlegend=True,
                        legend=dict(
                            orientation="v",
                            xanchor="left",
                            x=0.03,  
                            yanchor="top",
                            y=0.9,  
                            itemsizing='trace',  # To ensure items in legend keep the same size
                            traceorder="normal",
                            bgcolor="White",  # Background color
                            bordercolor="Black",  # Border color
                            borderwidth=1,  # Border width
                            groupclick="toggleitem",
                            itemclick="toggleothers",
                            itemdoubleclick="toggle",
                            tracegroupgap=100,
                            font_size=legend_font_size
                        )
)

In [ ]:
exps2plot = ['nsga2__anytown_mixed_f1__exp02', 'nsga2__anytown_rehab_f1__exp04', 'nsga2__anytown_rehab_f1__fullpower', 'nsga2__anytown_rehab_f1__median']
colors = ['#29378A', '#808080', '#74BDA7','#02B9EA']
markers = ['circle', 'circle', 'square', 'diamond']
names = ['Integrated', 'Pure design - Siew et al.', 'Pure design - Full power', 'Pure design - Median pattern']

def filter_pf(experiment: pybeme.Experiment):
    final_fvs = experiment.final_fitness_vectors.to_numpy()
    pf = pg.non_dominated_front_2d(final_fvs)
    pf = pf[final_fvs[pf,0] < 20e6]
    pf = pf[final_fvs[pf,1] < 0]
    return pf

def plot_pareto_fronts(experiments: dict, exps2plot: list, colors: list, markers: list, names: list):
    fig = go.Figure()

    for e, expname, in enumerate(exps2plot):
        final_fvs = experiments[expname].final_fitness_vectors.to_numpy()
        pf = filter_pf(experiments[expname])
        npf = np.setdiff1d(np.arange(final_fvs.shape[0]), pf)
        
        fig.add_trace(go.Scatter(x=final_fvs[pf,0], y=-final_fvs[pf,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e]),  
                                showlegend=True, name=names[e],
                                legendgroup=expname,
                                customdata=np.array(pf, dtype=str),
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                                ) )
        fig.add_trace(go.Scatter(x=final_fvs[npf,0], y=-final_fvs[npf,1], mode='markers', opacity=0.5,
                                marker=dict(size=msize, symbol=markers[e]+'-open', color=colors[e]),  
                                showlegend=True, name=names[e],
                                legendgroup=expname,
                                customdata=np.array(npf, dtype=str),
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                                ) )

    fix_layout(fig, 'Pareto fronts of the different solutions')

    fig.show()

plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

There is something wrong, we checked and I forgot to track the missing steps. This is not ok but it is the default behaviour with wntr...
with EPYT this is not hte the behaviour if u use getComputedHydraulicTimeSeries.

## v 24.06.0
We fixed the use of all the instances. now we have the correct reliability

In [ ]:
exps2plot = ['nsga2__anytown_mixed_f1__exp02', 'nsga2__anytown_mixed_f1__exp03', 'nsga2__anytown_rehab_f1__exp03', 'nsga2__anytown_rehab_f1__exp05']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated - v24.4.0', 'Integrated - v24.6.0', 'Pure design - v24.4.0', 'Pure design - v24.6.0']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)    

def percentage_errors(pre, post):
    pe = (pre - post) / np.abs(pre)
    # Failed solution have rel indx >= 0
    pe[post[:,1] >= 0, :] = -0.99
    return pe

versions2test = ['v24.01.00', 'v24.06.00']

def version_title_matrix(vers2test: list) -> list:
    assert len(vers2test) == 2, 'Only two versions are supported'
    titles = ["", f"{vers2test[0]} -> {vers2test[1]}", "", f"{vers2test[1]} -> {vers2test[0]}", "", "", "", ""]
    return titles

def plot_version_mismatch():
    fig = subp.make_subplots(rows=2, cols=4, subplot_titles=version_title_matrix(versions2test),
                            shared_xaxes=True, shared_yaxes=True,
                            vertical_spacing=0.01, horizontal_spacing=0.01,
                            row_heights=[0.8, 0.2], column_widths=[0.1, 0.4, 0.1, 0.4],
                            x_title="Evaluation version")
    fig.update_layout(width=1200, height=600)
    for c in range(1,5):
        for r in range(1, 3):
            fig.update_xaxes(showline=True, showgrid=True, linewidth=1, linecolor='grey', zerolinecolor='black', gridcolor='lightgrey', mirror=True,
                             range=[-1.01, 1.01],
                             row=r, col=c)
            fig.update_yaxes(showline=True, showgrid=True, linewidth=1, linecolor='grey', zerolinecolor='black', gridcolor='lightgrey', mirror=True,
                              range=[-1.01, 1.01],
                              row=r, col=c)
            fig.update_layout(plot_bgcolor='white', paper_bgcolor='white') 
    fig.update_xaxes(title_text='Cost percentage error', row=2, col=2)
    fig.update_xaxes(title_text='Cost percentage error', row=2, col=4)
    fig.update_yaxes(title_text='RelIdx percentage error', row=1, col=1)

    for e, expname in enumerate(exps2plot):
        final_fvs = experiments[expname].final_fitness_vectors
        pf_idx = filter_pf(experiments[expname])
        pf_pre = final_fvs.to_numpy()[pf_idx]
        final_indvs_coords = final_fvs.index[pf_idx]

        pf_post = np.zeros_like(pf_pre)
        for i, fic in enumerate(final_indvs_coords):
            final_indv_coord = (fic[0], # island name
                                experiments[expname].generations[fic[0]].to_numpy()[-1], # last generation of the island
                                fic[1]) # individual index

            sim = experiments[expname].simulator(final_indv_coord)
            
            if e % 2 == 0:
                valt = versions2test[1]
                valt_column = 2
            else:
                valt = versions2test[0]
                valt_column = 4
            
            sim.data['bemelib_version'] = valt
            sim.run()
            pf_post[i] = sim.result

        pe = percentage_errors(pf_pre, pf_post)
        pe_sux = pe[pe[:,1] != -0.99, :]
        fig.add_trace(go.Scatter(x=pe_sux[:,0], y=pe_sux[:,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e]),
                                showlegend=False,
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f}'
                                ), row=1, col=valt_column)
        pe_fail = pe[pe[:,1] == -0.99, :]
        fig.add_trace(go.Scatter(x=pe_fail[:,0], y=pe_fail[:,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e], line=dict(color='red', width=1)),  
                                showlegend=False,
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f}'
                                ), row=1, col=valt_column)
        
        # Add the distribution of the cost error, in the same column, but second row
        deltac_counts, deltac_bins = np.histogram(pe[:,0], bins=np.linspace(-1, 1, 20))
        fig.add_trace(go.Bar(x=deltac_bins+((deltac_bins[1]-deltac_bins[0])/2), y=deltac_counts/np.sum(deltac_counts), width=(deltac_bins[1]-deltac_bins[0]),
                            opacity=0.75,
                            marker=dict(color=colors[e]), showlegend=False), row=2, col=valt_column)

        # Add the distribution of the reliability index error, in the first row, but one column before
        deltar_counts, deltar_bins = np.histogram(pe[:,1], bins=np.linspace(-1, 1, 20))
        fig.add_trace(go.Bar(y=deltar_bins+((deltar_bins[1]-deltar_bins[0])/2), x=deltar_counts/np.sum(deltar_counts), width=(deltar_bins[1]-deltar_bins[0]), orientation='h',
                            opacity=0.75,
                            marker=dict(color=colors[e]), showlegend=False), row=1, col=valt_column-1)

        # Add the centroid of the error distribution
        fig.add_trace(go.Scatter(x=[np.mean(pe[:,0])], y=[np.mean(pe[:,1])], mode='markers',
                                marker=dict(size=2*msize, symbol='x-open', color=colors[e], line=dict(color=colors[e], width=1)),  
                                showlegend=False,
                                customdata=[np.mean(pe[:,0]), np.mean(pe[:,1])],
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                                ), row=1, col=valt_column)
    
    fig.update_layout(barmode='overlay')
    fig.show()

plot_version_mismatch()

Ok but still not happy so we decided to do shorter hydraulics because we were experimenting the same behavoiur 

In [ ]:
exps2plot = ['nsga2__anytown_mixed_f1__exp03', 'nsga2__anytown_mixed_f1__exp04', 'nsga2__anytown_mixed_f1__exp05',
             'nsga2__anytown_rehab_f1__exp05', 'nsga2__anytown_rehab_f1__exp06', 'nsga2__anytown_rehab_f1__exp07']
colors = ['#29378A', '#29378A', '#29378A',
          '#808080', '#808080', '#808080']
markers = ['circle', 'square', 'diamond',
           'circle', 'square', 'diamond']
names = ['Integrated - 60min', 'Integrated - 30min', 'Integrated - 15min',
        'Pure design - 60min', 'Pure design - 30min', 'Pure design - 15min']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

fig = subp.make_subplots(rows=3, cols=3, subplot_titles=['15 min -> 60 min', '30 min -> 60 min', '60 min -> 60 min',
                                                         '15 min -> 30 min', '30 min -> 30 min', '60 min -> 30 min',
                                                        '15 min -> 15 min', '30 min -> 15 min', '60 min -> 15 min'],
                            shared_xaxes=True, shared_yaxes=True)

for e, expname in enumerate(exps2plot):
    final_fvs = experiments[expname].final_fitness_vectors
    fv_pre = final_fvs.to_numpy()
    pf = pg.non_dominated_front_2d(fv_pre)
    pf = pf[fv_pre[pf,1] <= -0.1]
    pf_pre = fv_pre[pf]
    
    final_indvs_coords = final_fvs.index[pf]
    fv_post15 = np.zeros_like(pf_pre)
    fv_post30 = np.zeros_like(pf_pre)
    fv_post60 = np.zeros_like(pf_pre)

    for i, fic in enumerate(final_indvs_coords):
        final_indv_coord = (fic[0], # island name
                            experiments[expname].generations[fic[0]].to_numpy()[-1], # last generation of the island
                            fic[1]) # individual index

        sim = experiments[expname].simulator(final_indv_coord)
        
        for m, min in enumerate([15, 30, 60]):
            
            sim.run()

            if v == 0:
                fv_post_04[i] = sim.result
            else:
                fv_post_06[i] = sim.result
                
    # Now plot the differences
    if e == 0 or e == 2:
        col = 1
    else :
        col = 2

    delta_c_04 = fv_pf[:, 0] -fv_post_04[:, 0]
    delta_r_04 = fv_pf[:, 1] -fv_post_04[:, 1]
    delta_c_06 = fv_pf[:, 0] -fv_post_06[:, 0]
    delta_r_06 = fv_pf[:, 1] -fv_post_06[:, 1]

    # Percentage error, I NEED to keep the sign to check if we overestimate or underestimate
    pe_c_04= delta_c_04 / fv_pf[:, 0]
    pe_r_04 = delta_r_04 / -fv_pf[:, 1]
    pe_c_06 = delta_c_06 / fv_pf[:, 0]
    pe_r_06 = delta_r_06 / -fv_pf[:, 1]
    # Any reliability change greater in abs term than 1 is to consider a solution that was working that now is not working
    # We put them in -1.1 or percentage error 110%
    pe_r_04[np.abs(delta_r_04) > 1] = -1.1
    pe_r_06[np.abs(delta_r_06) > 1] = -1.1

    
    fig.add_trace(go.Scatter(x= pe_c_04, y= pe_r_04, mode='markers',
                            marker=dict(size=msize, symbol=markers[e], color=colors[e]),  
                            showlegend=False,
                            customdata=np.array(np.arange(fv_pre.shape[0]), dtype=str),
                            hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                            ), row=2, col=col)
    fig.add_trace(go.Scatter(x= pe_c_06, y= pe_r_06, mode='markers',
                            marker=dict(size=msize, symbol=markers[e], color=colors[e]),  
                            showlegend=False,
                            customdata=np.array(np.arange(fv_pre.shape[0]), dtype=str),
                            hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                            ), row=1, col=col)
    

    
# For all 4 subplots, fix the xrange and yrange between the calculated max and min
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(range=[-0.2, 0.2], row=i, col=j)
        fig.update_yaxes(range=[-1.2, 1.2], row=i, col=j)
fig.show()

In [ ]:
exps2plot = ['nsga2__anytown_mixed_f1__exp05', '017', 'nsga2__anytown_rehab_f1__exp05', '016']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated - Existing Pipe F1', 'Integrated - Existing Pipe F2', 'Pure design - Existing Pipe F1', 'Pure design - Existing Pipe F2']
legendgroup_titles = ['Integrated', 'Integrated', 'Pure design', 'Pure design']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

# Plot the HV to see if the f2 works better than f1. One HV value per each generation. Do it for each island
fig = go.Figure()
hv_refpoint = [0,0]
linestyles = ['solid', 'dashdot', 'solid', 'dashdot']
for e, expname in enumerate(exps2plot):
    genz = experiments[expname].generations
    fvs = experiments[expname].fitness_vectors
    # I will draw one line for each island
    for i, isl in enumerate(list(experiments[expname].data['archipelago']['islands'].keys())):
        n_reps = genz[isl].shape[0]
        hv = np.zeros(genz[isl].shape[0])
        for g, gen in enumerate(genz[isl]):
            datapoint = fvs.loc[isl].loc[gen].to_numpy()
            datapoint = datapoint[np.logical_and(datapoint[:,0]>0, datapoint[:,1]<0, datapoint[:,0] <= 20e6), :]
            if (datapoint.shape[0] > 0):
                datapoint[:,0] = -datapoint[:,0]/20e6
                hv[g] = pg.hypervolume(datapoint).compute(hv_refpoint)

        fig.add_trace(go.Scatter(x=genz[isl], y=hv, mode='lines',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e]),
                                line=dict(width=2, dash=linestyles[e]),
                                showlegend=True, name=isl,
                                legendgroup=expname,
                                legendgrouptitle_text=names[e],
                                customdata=np.array(np.arange(genz[isl].shape[0]), dtype=str),
                                hovertemplate='Generation: %{x} <br> HV: %{y:.2f} <extra>%{customdata}</extra>'
                                ))

#fix_layout(fig, 'Hypervolume of the different solutions')
fig.update_layout(legend=dict(groupclick="toggleitem"))
fig.show()

So we have a double benefit with this formulation:
1. in the integrated, more seeds converge.
2. in both, we don't really converge sooner, but we started from more feasible solutions...

In [ ]:
# Same as before. We need to check v24.10.0 vs 24.11.0 for the mixed and rehab solutions
exps2plot = ['017', '019', '016', '018']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated - v24.10.0', 'Integrated - v24.11.0', 'Pure design - v24.10.0', 'Pure design - v24.11.0']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

versions2test = ['v24.10.00', 'v24.11.00']
plot_version_mismatch()

Nothing changed, as I was expecting. 

## v24.12.0 aka inverted riser

In [ ]:
# Same as before. We need to check v24.11.0 vs 24.12.0 for the mixed and rehab solutions
exps2plot = ['019', '025', '018', '024']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated - v24.10.0', 'Integrated - v24.12.0', 'Pure design - v24.10.0', 'Pure design - v24.12.0']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

versions2test = ['v24.11.00', 'v24.12.00']
plot_version_mismatch()

In [ ]:
# Same as before. We need to check v24.12.0 vs 25.02.0 for the mixed and rehab solutions
exps2plot = ['025', '031', '024', '030']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated - v24.12.0', 'Integrated - v25.02.0', 'Pure design - v24.12.0', 'Pure design - v25.02.0']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

versions2test = ['v24.12.00', 'v25.02.00']
plot_version_mismatch()


In [ ]:
def plot_formulation_mismatch():
    fig = subp.make_subplots(rows=2, cols=4, subplot_titles=version_title_matrix(formulations2test),
                            shared_xaxes=True, shared_yaxes=True,
                            vertical_spacing=0.01, horizontal_spacing=0.01,
                            row_heights=[0.8, 0.2], column_widths=[0.1, 0.4, 0.1, 0.4],
                            x_title="Evaluation Formulation")
    fig.update_layout(width=1200, height=600)
    for c in range(1,5):
        for r in range(1, 3):
            fig.update_xaxes(showline=True, showgrid=True, linewidth=1, linecolor='grey', zerolinecolor='black', gridcolor='lightgrey', mirror=True,
                             range=[-1.01, 1.01],
                             row=r, col=c)
            fig.update_yaxes(showline=True, showgrid=True, linewidth=1, linecolor='grey', zerolinecolor='black', gridcolor='lightgrey', mirror=True,
                              range=[-1.01, 1.01],
                              row=r, col=c)
            fig.update_layout(plot_bgcolor='white', paper_bgcolor='white') 
    fig.update_xaxes(title_text='Cost percentage error', row=2, col=2)
    fig.update_xaxes(title_text='Cost percentage error', row=2, col=4)
    fig.update_yaxes(title_text='RelIdx percentage error', row=1, col=1)

    for e, expname in enumerate(exps2plot):
        final_fvs = experiments[expname].final_fitness_vectors
        pf_idx = filter_pf(experiments[expname])
        pf_pre = final_fvs.to_numpy()[pf_idx]
        final_indvs_coords = final_fvs.index[pf_idx]

        pf_post = np.zeros_like(pf_pre)
        for i, fic in enumerate(final_indvs_coords):
            final_indv_coord = (fic[0], # island name
                                experiments[expname].generations[fic[0]].to_numpy()[-1], # last generation of the island
                                fic[1]) # individual index

            sim = experiments[expname].simulator(final_indv_coord)

            # Convert from one formulation to another:
            if e % 2 == 0:
                sim.convert_to_formulation(formulations2test[1])
                valt_column = 2
            else:
                sim.convert_to_formulation(formulations2test[0])
                valt_column = 4
            
            sim.run()
            pf_post[i] = sim.result

        pe = percentage_errors(pf_pre, pf_post)
        pe_sux = pe[pe[:,1] != -0.99, :]
        fig.add_trace(go.Scatter(x=pe_sux[:,0], y=pe_sux[:,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e]),
                                showlegend=False,
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f}'
                                ), row=1, col=valt_column)
        pe_fail = pe[pe[:,1] == -0.99, :]
        fig.add_trace(go.Scatter(x=pe_fail[:,0], y=pe_fail[:,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e], line=dict(color='red', width=1)),  
                                showlegend=False,
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f}'
                                ), row=1, col=valt_column)
        
        # Add the distribution of the cost error, in the same column, but second row
        deltac_counts, deltac_bins = np.histogram(pe[:,0], bins=np.linspace(-1, 1, 20))
        fig.add_trace(go.Bar(x=deltac_bins+((deltac_bins[1]-deltac_bins[0])/2), y=deltac_counts/np.sum(deltac_counts), width=(deltac_bins[1]-deltac_bins[0]),
                            opacity=0.75,
                            marker=dict(color=colors[e]), showlegend=False), row=2, col=valt_column)

        # Add the distribution of the reliability index error, in the first row, but one column before
        deltar_counts, deltar_bins = np.histogram(pe[:,1], bins=np.linspace(-1, 1, 20))
        fig.add_trace(go.Bar(y=deltar_bins+((deltar_bins[1]-deltar_bins[0])/2), x=deltar_counts/np.sum(deltar_counts), width=(deltar_bins[1]-deltar_bins[0]), orientation='h',
                            opacity=0.75,
                            marker=dict(color=colors[e]), showlegend=False), row=1, col=valt_column-1)

        # Add the centroid of the error distribution
        fig.add_trace(go.Scatter(x=[np.mean(pe[:,0])], y=[np.mean(pe[:,1])], mode='markers',
                                marker=dict(size=2*msize, symbol='x-open', color=colors[e], line=dict(color=colors[e], width=1)),  
                                showlegend=False,
                                customdata=[np.mean(pe[:,0]), np.mean(pe[:,1])],
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                                ), row=1, col=valt_column)
    
    fig.update_layout(barmode='overlay')
    fig.show()

In [ ]:
# Same as before. We need to check v25.02.0 vs 25.03.0 for the mixed and rehab solutions
exps2plot = ['030', '032', '031', '033']
colors = ['#808080', '#808080', '#29378A', '#29378A']
markers = ['circle', 'square', 'circle', 'square']
names = ['Pure design :: f2', 'Pure design :: f3', 'Integrated :: f2', 'Integrated :: f3']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

formulations2test = [2, 3]

plot_formulation_mismatch()